In [1]:
from langchain.chains import RetrievalQA
from langchain.vectorstores import DocArrayInMemorySearch
from IPython.display import display, Markdown
from langchain.prompts import ChatPromptTemplate
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain, SequentialChain
import openai

class configs():
    openai_api_key= "your openai api-key"
    openai.api_key =openai_api_key
    openai.api_base="https://api.closeai-proxy.xyz/v1"
    # path to paper
    pdf_path = "1.pdf"
    # path to code
    code_path = "code.txt"
    # iteration number
    iterations = 3

In [13]:
# Define LLM-enhanced multi-agent system
class LLM_enhanced_multi_agent_system_for_SC():
    def __init__(self, args:configs):
        self.api_key = args.openai_api_key
        self.llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.9, openai_api_key=self.api_key)
        self.iterations = args.iterations
        self.embeddings = OpenAIEmbeddings(openai_api_key=self.api_key)

    # Validate the legitimacy of the input
    def secure_agent(self,query):
        response = openai.Moderation.create(
        input=query,
        )
        moderation_output = response["results"][0]
        print(moderation_output)
        if moderation_output["flagged"]:
            print("Illegal input!")
            return False
        else:
            return query

    # Read the paper and generate vector memory
    def condensate_agent(self, pdf_path=""):
        loader = PyPDFLoader(pdf_path)
        docs = loader.load()
        self.db = DocArrayInMemorySearch.from_documents(
            docs,
            self.embeddings
        )
    # Distill relevant SC knowledge for constructing the SC model
    def inference_agent(self,query):
        retriever = self.db.as_retriever()
        qa_stuff = RetrievalQA.from_chain_type(
            llm=self.llm,
            chain_type="stuff",# Stuff means merging text fragments into one paragraph of text
            retriever=retriever,
            verbose=True
        )
        response = qa_stuff.run(query)
        display(Markdown(response))
        return response

    # Formulates a specific sub-task chain for the SC model
    def planning_agent(self, input_name="paper_ref", output_name="sc_scheme"):
        scheme_prompt = ChatPromptTemplate.from_template(
            "# Referring to the following description of the semantic communication model: '''{"
            f"{input_name}"
            "}'''."
            "# Output the design scheme for achieving the semantic communication model."
        )
        chain = LLMChain(llm=self.llm, prompt=scheme_prompt,
                             output_key=output_name,
                             )
        return chain
    # sub-task chain for code generation
    def code_generation(self,input_name="sc_scheme", output_name="sc_model"):
        data = [
            "Hello there!",
            "How are you doing?",
            "PyTorch is great.",
            "Machine learning is fascinating.",
            "Let's build something amazing.",
            "Semantic communication matters.",
            "Understanding context is important.",
            "AI is changing the world.",
            "Keep learning and growing.",
            "Innovation drives progress."
        ]
        model_prompt = ChatPromptTemplate.from_template(
            "# According to the design scheme of the semantic communication model: '''{"
            f"{input_name}"
            "}'''."
            "# Refering to the reference code: '''{ref_code}''',"
            "output the Pytorch framework-based Python code for achieving the semantic communication model, "
            f"assuming the input data is '''{data}''', which is a list of sentences. Additionally, the total number of model parameters does not exceed 2,000,000."
        )
        chain = LLMChain(llm=self.llm, prompt=model_prompt,
                             output_key=output_name,
                             )
        return chain

    # evaluate the quality of the generated code
    def evaluate_agent(self, input_name="sc_model", output_name="eval_score"):
        # Manually input bleu scores for SC model
        eval_prompt = ChatPromptTemplate.from_template(
            "The quality of the generated code is evaluated and an overall score is given from three aspects: "
            "1. Quality of code, including its reasonableness and completeness."
            "2. The performance of code, using bleu score as the evaluation indicator."
            "3. Whether the number of parameters in the generated code meets the parameters requirement."
            "Based on the above requirements, evaluate the quality of the 'generated code':'''{"
            f"{input_name}"
            "}'''."
            "## Output the evaluated results. The format of the output refers to:"
            "```json Dict('model parameters': int \ the model parameters of the 'generated code', 'evaluated score': int \ The evaluated score is based on the performance of the reference code in three aspects. The evaluated score ranges from 0 to 100 and is not the same as the last time."
            "'evaluation': string \ Based on the performance of the reference code in three aspects, give the reviews, including the model parameters, the description of 'generated code', the advantage and disadvantage analysis of the 'generated code', etc.)```"
            )
        chain = LLMChain(llm=self.llm, prompt=eval_prompt,
                                 output_key=f"{output_name}",
                                 )
        return chain

    # Improve the generated codes according to suggestion
    def reflexion_and_refinement_agents(self,inputs, output_name="sc_model"):
        model_prompt = ChatPromptTemplate.from_template(
        "# Modify the Python code: '''{"
        f"{inputs[0]}"
        "}'''."
        "# According to the corresponding evaluation results:'''{"
        f"{inputs[1]}"
        "}'''."
        "# If the 'evaluated score'< 60, a major revision is required (e.g., change the architecture of the networks); otherwise, it is considered a minor revision (e.g., change the layers of the networks)."
        "# Output the modified Python codes that aim to improve the 'evaluated score' and obtain a score of no less than 90 finally."
        )
        chain = LLMChain(llm=self.llm, prompt=model_prompt,
                             output_key=output_name,
                             )
        return chain

    # Combine all agents for SC system generation
    def create_chains(self):
        chains = []
        output_keys = []
        # define sc scheme 1
        input_name = "paper_ref"
        output_name = "scheme_1"
        chain = self.planning_agent(input_name,output_name)
        chains.append(chain)
        output_keys.append(output_name)
        # define sc scheme 2
        input_name = "paper_ref"
        output_name = "scheme_2"
        chain = self.planning_agent(input_name, output_name)
        chains.append(chain)
        output_keys.append(output_name)
        # define code generation 1
        input_name = "scheme_1"
        output_name = "sc_model_0_1"
        chain = self.code_generation(input_name,output_name)
        chains.append(chain)
        output_keys.append(chain.output_key)

        # define code generation 2
        input_name = "scheme_2"
        output_name = "sc_model_0_2"
        chain = self.code_generation(input_name,output_name)
        chains.append(chain)
        output_keys.append(chain.output_key)
        for i in range(self.iterations):
            # eval sc model
            input_name = f"sc_model_{i}_1"
            output_name = f"eval_score_{i}_1"
            chain = self.evaluate_agent(input_name,output_name)
            chains.append(chain)
            output_keys.append(chain.output_key)

            input_name = f"sc_model_{i}_2"
            output_name = f"eval_score_{i}_2"
            chain = self.evaluate_agent(input_name,output_name)
            chains.append(chain)
            output_keys.append(chain.output_key)

            # improve sc model
            inputs = [f"sc_model_{i}_1",f"eval_score_{i}_1"]
            output_name = f"sc_model_{i+1}_1"
            chain = self.reflexion_and_refinement_agents(inputs,output_name)
            chains.append(chain)
            output_keys.append(chain.output_key)

            inputs = [f"sc_model_{i}_2",f"eval_score_{i}_2"]
            output_name = f"sc_model_{i+1}_2"
            chain = self.reflexion_and_refinement_agents(inputs,output_name)
            chains.append(chain)
            output_keys.append(chain.output_key)

        # eval the final sc model
        input_name = f"sc_model_{self.iterations}_1"
        output_name = f"eval_score_{self.iterations}_1"
        chain = self.evaluate_agent(input_name,output_name)
        chains.append(chain)
        output_keys.append(chain.output_key)

        input_name = f"sc_model_{self.iterations}_2"
        output_name = f"eval_score_{self.iterations}_2"
        chain = self.evaluate_agent(input_name,output_name)
        chains.append(chain)
        output_keys.append(chain.output_key)

        self.overall_chain = SequentialChain(
            chains=chains,
            input_variables=["paper_ref","ref_code"],
            output_variables=output_keys,
            verbose=True,
        )

    # Run multi-agent system
    def Run(self, inputs):
        outputs = self.overall_chain(inputs)
        return outputs

In [14]:
# generate sc by LLM enhanced multi-agent system
args = configs()
ref_code = open(args.code_path,"r",encoding="utf-8").read()
LL_SC = LLM_enhanced_multi_agent_system_for_SC(args)
LL_SC.condensate_agent(args.pdf_path)
paper_query = "Composition of the semantic communication model"
if LL_SC.secure_agent(paper_query):
    SC_Components = LL_SC.inference_agent(paper_query)
    LL_SC.create_chains()
    results = LL_SC.Run({"paper_ref":SC_Components,"ref_code":ref_code})
else:
    pass

{
  "categories": {
    "harassment": false,
    "harassment/threatening": false,
    "hate": false,
    "hate/threatening": false,
    "self-harm": false,
    "self-harm/instructions": false,
    "self-harm/intent": false,
    "sexual": false,
    "sexual/minors": false,
    "violence": false,
    "violence/graphic": false
  },
  "category_scores": {
    "harassment": 0.0001638556132093072,
    "harassment/threatening": 5.502765816345345e-06,
    "hate": 0.0031374176032841206,
    "hate/threatening": 3.2831585485837422e-06,
    "self-harm": 4.89498279421241e-06,
    "self-harm/instructions": 1.3621797734231222e-06,
    "self-harm/intent": 6.14288126143947e-07,
    "sexual": 0.0002470398903824389,
    "sexual/minors": 5.0377188017591834e-05,
    "violence": 0.00011020625242963433,
    "violence/graphic": 5.424775736173615e-05
  },
  "flagged": false
}


> Entering new RetrievalQA chain...

> Finished chain.


The semantic communication model typically comprises a semantic encoder, channel encoder, channel decoder, semantic decoder, and a knowledge base. The semantic encoder extracts semantic information from the original data and encodes it into semantic features. The channel encoder encodes and modulates the semantic features to ensure data transmission on the physical channel. The channel decoder de-modulates and decodes the received signal. The semantic decoder aims to understand the received semantic features and recover the original data. Finally, the knowledge base is a universal knowledge model that helps the semantic encoder and decoder understand and infer semantic information effectively.



> Entering new SequentialChain chain...

> Finished chain.


In [15]:
results['paper_ref']

'The semantic communication model typically comprises a semantic encoder, channel encoder, channel decoder, semantic decoder, and a knowledge base. The semantic encoder extracts semantic information from the original data and encodes it into semantic features. The channel encoder encodes and modulates the semantic features to ensure data transmission on the physical channel. The channel decoder de-modulates and decodes the received signal. The semantic decoder aims to understand the received semantic features and recover the original data. Finally, the knowledge base is a universal knowledge model that helps the semantic encoder and decoder understand and infer semantic information effectively.'

In [16]:
results['sc_model_1_1']

'Since the evaluated score is already 75, it falls under the category of minor revision. To improve the score further and aim for a score of no less than 90, we can make the following modifications:\n\n1. Increase the complexity of the model by adding more layers or increasing the hidden size of the existing layers.\n2. Experiment with different optimization algorithms or learning rates to enhance training efficiency.\n3. Explore techniques like teacher forcing, attention mechanisms, or beam search to improve decoding accuracy.\n\nHere is an example of modifying the existing code by increasing the hidden size of the Semantic Encoder and Semantic Decoder to improve the model\'s capacity:\n\n```python\n# Modify the hidden size of the Semantic Encoder and Semantic Decoder\nclass SemanticEncoder(nn.Module):\n    def __init__(self, vocab_size, embedding_size, hidden_size):\n        super(SemanticEncoder, self).__init__()\n        self.embedding = nn.Embedding(vocab_size, embedding_size)\n  

In [21]:
results['eval_score_3_1']

'```json\n{\n  "model parameters": "Advanced techniques like attention mechanisms, regularization techniques, and fine-tuning of hyperparameters",\n  "evaluated score": 92,\n  "evaluation": "The provided updated Python code has successfully incorporated advanced techniques to improve model performance, achieving an impressive evaluated score of 92. The code surpasses the target score of no less than 90, showcasing its effectiveness in enhancing model performance. With no further modifications required, the code demonstrates strong reasoning, completeness, and performance, meeting the parameters requirement and excelling in all evaluated aspects."\n}\n```'